## Climate App

* RICARDO REYNOSO RAMOS

In [1]:
from flask import Flask, redirect, jsonify
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime


In [2]:
#Setup database
#create engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
#reflect database and tables
Base = automap_base()
Base.prepare(engine, reflect = True)

In [4]:
#save table references
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
#create session
session = Session(engine)

In [6]:
#Setup Flask
climate_app = Flask(__name__)

In [7]:
#Establish the FLASK ROUTES
@climate_app.route("/")
def display():
    """List all available api routes."""
    return (
        f"Avalable Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"-the dates and precipitation observations from the last year<br/>"
        f"/api/v1.0/stations"
        f"- list of stations from the dataset<br/>"
        f"//api/v1.0/tobs"
        f"- list of Temperature Observations (tobs) for the previous year<br/>"
        f"/api/v1.0/calc_temps/<start>"
        f"- list of `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date<br/>"
        f"/api/v1.0/calc_temps/<start>/<end>"
        f"- the `TMIN`, `TAVG`, and `TMAX` for dates between the start and end date inclusive<br/>"
    )

In [8]:
@climate_app.route("/api/v1.0/precipitation")
def precipitations():
    """Return a list of dates and precipitation observations"""
 
    prcp_data = session.query(Measurement.date, Measurement.prcp).\
                   filter(Measurement.date.between('2016-08-01', '2017-08-01')).all()

    precipitations= []
    for result in prcp_data:
        row = {"date":"prcp"}
        row["date"] = result[0]
        row["prcp"] = float(result[1])
        precipitations.append(row)

    return jsonify(precipitations)

In [9]:
@climate_app.route("/api/v1.0/stations")
def stations():
   
    # Query all stations from the station table
    station_data = session.query(Station.station, Station.station_name).group_by(Station.station).all()

    station_final = list(np.ravel(station_data))
    return jsonify(station_final)

In [10]:
@climate_app.route("/api/v1.0/tobs")
def tobs():
    tobs_data = session.query(Measurement.station, Measurement.tobs).filter(Measurement.date.between('2016-08-01', '2017-08-01')).all()
    
    tobs_final=[]
    for tobs in tobs_data:
        tobs_dict = {}
        tobs_dict["station"] = tobs[0]
        tobs_dict["tobs"] = float(tobs[1])
       
        tobs_final.append(tobs_dict)
    return jsonify(tobs_final)

In [11]:
@climate_app.route("/api/v1.0/calc_temps/<start>")
def calcs_temps(start='start_date'):
    start_date = datetime.strptime('2016-08-01', '%Y-%m-%d').date()
    start_results = session.query(func.max(Measurement.tobs), \
                            func.min(Measurement.tobs),\
                            func.avg(Measurement.tobs)).\
                            filter(Measurement.date >= start_date)
    start_tobs = []
    for tobs in start_results:
        tobs_dict = {}
        tobs_dict["TAVG"] = float(tobs[2])
        tobs_dict["TMAX"] = float(tobs[0])
        tobs_dict["TMIN"] = float(tobs[1])
        
        start_tobs.append(tobs_dict)

    return jsonify(start_tobs)


In [12]:
@climate_app.route("/api/v1.0/calc_temps/<start>/<end>")
def calcs_temps_b(start='start_date', end='end_date'):      
    start_date = datetime.strptime('2016-08-01', '%Y-%m-%d').date()
    end_date = datetime.strptime('2017-08-01', '%Y-%m-%d').date()

    start_end_results=session.query(func.max(Measurement.tobs).label("max_tobs"), \
                      func.min(Measurement.tobs).label("min_tobs"),\
                      func.avg(Measurement.tobs).label("avg_tobs")).\
                      filter(Measurement.date.between(start_date , end_date))   
    start_end_tobs = []
    for tobs in start_end_results:
        tobs_dict = {}
        tobs_dict["TAVG"] = float(tobs[2])
        tobs_dict["TMAX"] = float(tobs[0])
        tobs_dict["TMIN"] = float(tobs[1])

        start_end_tobs.append(tobs_dict)
    
    return jsonify(start_end_tobs)

In [14]:
if __name__ == '__main__':
    climate_app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1